In [2]:
import os
import openai
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [6]:
llm = OpenAI(model_name="text-ada-001")
llm("Tell me about today's climate in vizag")

'\n\nThe climate in vizag is different every day. It is very hot which makes it difficult to stay alive in the cold weather. There is a lot of rain every day which makes it difficult to stay alive in the cold weather. The snow is not very present in the atmosphere and makes it very difficult to stay alive in the cold weather.'

In [7]:
template = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}
"""

In [8]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template=template)
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.predict(input="I like this waterfalls and its looking lovely!")

'\nWaterfalls: I like this\nsubject: Waterfalls\n\ntext: I like this waterfalls and its looking lovely!\n\nWaterfalls: I like this'

In [15]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

sentiment_schema = ResponseSchema(
    name="sentiment",
    description="Is the text positive, neutral or negative? Only provide these words",
)
subject_schema = ResponseSchema(
    name="subject", description="What subject is the text about? Use exactly one word."
)
response_schemas = [sentiment_schema, subject_schema]

In [16]:
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"sentiment": string  // Is the text positive, neutral or negative? Only provide these words\n\t"subject": string  // What subject is the text about? Use exactly one word.\n}\n```'

In [17]:
template = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Just return the JSON, do not add ANYTHING, NO INTERPRETATION!

text: {input}

{format_instructions}

"""

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_template(template=template)

format_instructions = parser.get_format_instructions()

messages = prompt.format_messages(
    input="I like this waterfalls and its looking lovely!",
    format_instructions=format_instructions,
)

chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
response

AIMessage(content='```json\n{\n\t"sentiment": "positive",\n\t"subject": "waterfalls"\n}\n```', additional_kwargs={}, example=False)

In [18]:
output_dict = parser.parse(response.content)
output_dict

{'sentiment': 'positive', 'subject': 'waterfalls'}